In [ ]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.applications.vgg16 import VGG16
from keras.layers import Flatten, Dense
from keras.models import Model
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import json
import os
import cv2
import numpy as np

# 이미지와 JSON 파일의 경로
image_dir = '/Users/bumsoojoe/Desktop/팀프로젝트2_교통법규/test/toy_data/image'
json_dir = '/Users/bumsoojoe/Desktop/팀프로젝트2_교통법규/test/toy_data/json'

images = []
annotations = []

for filename in os.listdir(image_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # 이미지 파일인 경우
        image_path = os.path.join(image_dir, filename)
        json_path = os.path.join(json_dir, filename.split('.')[0] + '.json')

        # 이미지 불러오기
        image = cv2.imread(image_path)
        images.append(image)

        # JSON 라벨 불러오기
        with open(json_path, 'r') as f:
            data = json.load(f)
            
            image_info = {
                "width": data["description"]["imageWidth"],
                "height": data["description"]["imageHeight"],
            }

            env_annotations = data["annotations"]["environment"]
            pm_annotations = data["annotations"]["PM"]
            
            annotation = {
                "image_info": image_info,
                "environment": env_annotations,
                "PM": pm_annotations
            }
            annotations.append(annotation)

labels = []
for annotation in annotations:
    if annotation['environment']:
        labels.append(0)
    elif annotation['PM']:
        labels.append(1)
    else:
        labels.append(-1)  # 다른 라벨 또는 기본값을 위한 코드



# 데이터 전처리
X = np.array(images)
Y = np.array(labels)
Y = tf.keras.utils.to_categorical(Y, num_classes=2)  # 클래스가 2개라고 가정

resized_images = [cv2.resize(img, (224, 224)) for img in images]
X = np.array(resized_images) / 255.0


# 3. 모델 학습
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(2, activation="softmax")(x)  # 2는 분류할 클래스의 수입니다. 필요에 따라 변경해야 합니다.
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(X, Y, epochs=10, batch_size=32)

# 4. 위반 사항 분석
new_img_path = "/Users/bumsoojoe/Desktop/팀프로젝트2_교통법규/Team_Project_2_Traffic/toy/toy_test/C001237_008_0220_C_N_F_0.jpg"
new_img = keras.preprocessing.image.load_img(new_img_path, target_size=(224, 224))
new_img_arr = np.expand_dims(keras.preprocessing.image.img_to_array(new_img), axis=0)

# 분류 예측
predictions = model.predict(new_img_arr)

# 결과 출력
if np.argmax(predictions) == 1:  # 위반 사항을 탐지한 경우
    print("교통법규 위반 사항이 감지되었습니다.")
else:
    print("교통법규 위반 사항이 없습니다.")